In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

## IMDB dataset and the sentiment classification task

In [2]:
PATH='data/aclImdb/'
names=['neg', 'pos']

In [3]:
%ls {PATH}

README      imdb.vocab  imdbEr.txt
